In [ ]:
import torch as T
import numpy as np
import gym

from IPython.display import clear_output
from Ractor_critic import ActorCritic

In [ ]:
from enviroment_class import CustomEnv

env = CustomEnv(obstacle_turn = False,
                vizualaze     = False,
                Total_war     = False,
                inp_dim       = 500,
                head_velocity = 0.005,#0.005
                num_obs       = 5,
                num_enemy     = 1,
                size_obs      = [50, 60],
                rew_col       = -70,
                rew_win       = 100,
                rew_defeat    = -100,
                steps_limit   = 1000,
                EnemyLidSet   = [45,90],
                AllyLidSet    = [40, 90])
_ = env.reset()


pygame 2.1.0 (SDL 2.0.16, Python 3.10.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/parkhomenko/PycharmProjects/RL/venv/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


In [ ]:
agent = ActorCritic(lr = 0.0005, 
                    input_dim  = env.observation_space['img'].shape,
                    n_actions  = env.action_space.n,
                    gamma = 0.96)

AttributeError: 'Discrete' object has no attribute 'high'

In [ ]:
scores = []
save_score = -np.inf

n_steps = 10
T_MAX = 10000

In [ ]:
for i in range(n_steps):
    observation = env.reset()
    
    done = False
    ep_steps = 0
    r = 0
    hx, cx = T.zeros(1,64).to(agent.actor.device), T.zeros(1,64).to(agent.actor.device)

    while not done:

        action, hx, cx = agent.choose_action(observation, hx, cx)
        observation, reward, done, info = env.step(action)
        agent.store_mem(reward)
        
        r += reward
        ep_steps+=1
        
        if ep_steps % T_MAX == 0 or done:
            hx, cx = hx.detach(), cx.detach()
            loss = agent.learn(observation, done)
            
    scores.append(r)
    
    avg_score = np.mean(scores[-100:])
    if avg_score > save_score:
        agent.save_models()
        save_score = avg_score
    
    if not bool(i%10):clear_output()
    print(i,' reward:', r,' avg score:', avg_score,' loss:', loss)

In [ ]:
# agent.load_models()

In [ ]:
import matplotlib.pyplot as plt
def plot_learning_curve(scores, x, figure_file):
    # loss_avg = np.zeros(len(scores))
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
        # loss_avg[i] = np.mean(loss[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    # plt.plot(x, loss_avg)
    plt.title('Average of previos 100 games')
    plt.savefig(figure_file)

In [ ]:
steps_list = [z for z in range(n_steps)]

plot_learning_curve(scores, steps_list, 'A3C_pong_final.png')